## **Augment Data**

In [25]:
!pip show librosa

Name: librosa
Version: 0.11.0
Summary: Python module for audio and music processing
Home-page: https://librosa.org
Author: Brian McFee, librosa development team
Author-email: brian.mcfee@nyu.edu
License: ISC
Location: C:\Users\Pras\miniconda3\Lib\site-packages
Requires: audioread, decorator, joblib, lazy_loader, msgpack, numba, numpy, pooch, scikit-learn, scipy, soundfile, soxr, standard-aifc, standard-sunau, typing_extensions
Required-by: audiomentations


In [26]:
!pip install pydub ffmpeg-python


In [27]:
# --- CELL 1: Import library ---
import os
import random
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from pydub import AudioSegment
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Gain


In [28]:
# --- CELL 2: Setting folder dataset dan target ---
base_dir = "dataset"                     # folder asal dataset
output_dir = "dataset_augmented"         # folder hasil augmentasi
target_samples = 100                     # target minimal per kelas

# Pastikan folder output ada
os.makedirs(output_dir, exist_ok=True)



In [29]:
# --- CELL 3: Definisi augmentasi ---
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-3, max_semitones=3, p=0.5),
    Gain(min_gain_db=-6, max_gain_db=6, p=0.5)
])

In [30]:
# --- CELL 4: Fungsi augmentasi per kelas ---
def augment_audio_class(folder_name):
    """
    folder_name: nama subfolder (misal 'buka' atau 'tutup')
    """
    src_folder = os.path.join(base_dir, folder_name)
    dst_folder = os.path.join(output_dir, folder_name)

    # Buat folder output jika belum ada
    os.makedirs(dst_folder, exist_ok=True)

    # Ambil daftar file
    files = [f for f in os.listdir(src_folder) if f.lower().endswith(('.wav', '.m4a'))]
    print(f"🎧 {folder_name}: {len(files)} file asli ditemukan")

    counter = 0
    # Lakukan augmentasi sampai total file mencapai target
    while len(os.listdir(dst_folder)) < target_samples:
        file = random.choice(files)
        path = os.path.join(src_folder, file)

        try:
            # === Load file ===
            if file.lower().endswith(".m4a"):
                audio = AudioSegment.from_file(path, format="m4a")
                signal = np.array(audio.get_array_of_samples()).astype(np.float32)
                signal = signal / np.iinfo(audio.array_type).max  # normalisasi -1..1
                sr = audio.frame_rate
            else:
                signal, sr = librosa.load(path, sr=None)

            # === Augmentasi ===
            augmented = augment(samples=signal, sample_rate=sr)

            # === Simpan hasil ===
            out_name = f"aug_{counter}_{os.path.splitext(file)[0]}.wav"
            out_path = os.path.join(dst_folder, out_name)
            sf.write(out_path, augmented, sr)
            counter += 1

            print(f"  ✅ {out_name} disimpan ({len(os.listdir(dst_folder))}/{target_samples})", end="\r")

        except Exception as e:
            print(f"⚠️ Gagal augmentasi {file}: {e}")

    print(f"\n➡️ Total file di {dst_folder}: {len(os.listdir(dst_folder))}")


In [31]:
# --- CELL 5: Jalankan augmentasi untuk kedua kelas ---
augment_audio_class("buka")
augment_audio_class("tutup")

print("\n✨ Augmentasi selesai! Hasil tersimpan di folder:", output_dir)

🎧 buka: 35 file asli ditemukan

➡️ Total file di dataset_augmented\buka: 100
🎧 tutup: 35 file asli ditemukan

➡️ Total file di dataset_augmented\tutup: 100

✨ Augmentasi selesai! Hasil tersimpan di folder: dataset_augmented


## **Ekstraksi Fitur (MFCC)**

In [32]:
def extract_features(folder, label):
    features = []
    for file in os.listdir(folder):
        if file.endswith((".wav", ".m4a")):
            path = os.path.join(folder, file)
            signal, sr = librosa.load(path, sr=None)
            mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
            mfcc_mean = np.mean(mfcc.T, axis=0)
            features.append([*mfcc_mean, label])
    return features


# --- Ekstraksi untuk kedua kelas ---
buka_features = extract_features("dataset_augmented/buka", "buka")
tutup_features = extract_features("dataset_augmented/tutup", "tutup")

# --- Gabungkan ke DataFrame ---
all_features = buka_features + tutup_features
columns = [f"mfcc_{i+1}" for i in range(13)] + ["label"]
df = pd.DataFrame(all_features, columns=columns)

# Simpan ke CSV untuk dipakai training
df.to_csv("fitur_audio.csv", index=False)
print("✅ Ekstraksi selesai, file tersimpan: fitur_audio.csv")

✅ Ekstraksi selesai, file tersimpan: fitur_audio.csv


## Split Data & Training Model

In [33]:
# --- Training Model ---
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Baca fitur
data = pd.read_csv("fitur_audio.csv")

X = data.drop("label", axis=1)
y = data["label"]

# Split data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Buat model dan latih
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# --- Evaluasi Model ---
y_pred = model.predict(X_test)
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\n📈 Classification Report:")
print(classification_report(y_test, y_pred))



📊 Confusion Matrix:
[[19  1]
 [ 0 20]]

📈 Classification Report:
              precision    recall  f1-score   support

        buka       1.00      0.95      0.97        20
       tutup       0.95      1.00      0.98        20

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97        40
weighted avg       0.98      0.97      0.97        40



## Simpan Model

In [ ]:
import joblib

# Simpan model
joblib.dump(model, "model/model_randomforest.pkl")
print("✅ Model disimpan ke file: model/model_randomforest.pkl")


✅ Model disimpan ke file: model/model_randomforest.pkl


In [35]:
def predict_audio(file_path, model):
    """
    Memprediksi apakah file audio termasuk 'buka' atau 'tutup'
    """
    if not os.path.exists(file_path):
        print(f"❌ File tidak ditemukan: {file_path}")
        return
    
    try:
        # --- Gunakan pydub jika format m4a ---
        if file_path.endswith(".m4a"):
            audio = AudioSegment.from_file(file_path, format="m4a")
            signal = np.array(audio.get_array_of_samples()).astype(np.float32)
            signal = signal / np.iinfo(audio.array_type).max
            sr = audio.frame_rate
        else:
            signal, sr = librosa.load(file_path, sr=None)

        # --- Ekstraksi MFCC ---
        mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc.T, axis=0).reshape(1, -1)

        # --- Prediksi ---
        pred = model.predict(mfcc_mean)[0]
        print(f"🎤 Prediksi untuk '{os.path.basename(file_path)}' → {pred}")

    except Exception as e:
        print(f"⚠️ Gagal memproses file {file_path}: {e}")


In [38]:
predict_audio("dataset/buka/Buka1.m4a", model)
predict_audio("dataset/tutup/tutup1.wav", model)


🎤 Prediksi untuk 'Buka1.m4a' → buka
🎤 Prediksi untuk 'tutup1.wav' → tutup


c:\Users\Pras\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Pras\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
